In [ ]:
# JobID of document analysis
JOBID= 'e65b673f999d48fdbfbb064a2b3ea62d9e9c57012997686442b9ea7231c914fa'

In [ ]:
# Config
REGION = aws_region = boto3.session.Session().region_name # 'us-west-2'

BUCKET = bucket_name = 'andrew-ocr'
document_name = 'jpg-raw/IMG_0102.JPG'

In [ ]:
import boto3
import io
import json
import uuid
import botocore
import time
import botocore

# Amazon SageMaker client
sagemaker = boto3.client('sagemaker', REGION)

# Amazon Textract client
textract = boto3.client('textract', REGION)

# S3 client
s3 = boto3.client('s3', REGION)

# A2I Runtime client
a2i_runtime_client = boto3.client('sagemaker-a2i-runtime', REGION)

In [ ]:
analyzePDF_results = textract.get_document_analysis(JobId=JOBID)
paginationToken= analyzePDF_results['NextToken']
analyzePDF_results2 = textract.get_document_analysis(JobId=JOBID, NextToken=paginationToken)

In [ ]:
paginationToken2= analyzePDF_results2['NextToken']
analyzePDF_results3 = textract.get_document_analysis(JobId=JOBID, NextToken=paginationToken2)

In [ ]:
from pprint import pprint

In [ ]:
# import pprint

# # Pretty print setup
# pp = pprint.PrettyPrinter(indent=2)

# # Function to pretty-print AWS SDK responses
# def print_response(response):
#     if 'ResponseMetadata' in response:
#         del response['ResponseMetadata']
#     pp.pprint(response)

In [ ]:
def get_csv(response):
    # Get the text blocks
    blocks=response['Blocks']
    pprint(blocks)

    blocks_map = {}
    table_blocks = []
    for block in blocks:
        blocks_map[block['Id']] = block
        if block['BlockType'] == "TABLE":
            table_blocks.append(block)

    if len(table_blocks) <= 0:
        return "<b> NO Table FOUND </b>"

    csv = ''
    for index, table in enumerate(table_blocks):
        csv += generate_table_csv(table, blocks_map, index +1)
        csv += '\n\n'

    return csv

In [ ]:
def GetResults(jobId):
    maxResults = 1000
    paginationToken = None
    finished = False
    pages= []
    while finished == False:
        response = None
#         if self.processType == ProcessType.ANALYSIS:
        if paginationToken == None:
#             response = self.textract.get_document_analysis(JobId=jobId, MaxResults=maxResults)
            response = textract.get_document_analysis(JobId=jobId)
        else:
#             response = self.textract.get_document_analysis(JobId=jobId, MaxResults=maxResults, NextToken=paginationToken)
            response = textract.get_document_analysis(JobId=jobId, NextToken=paginationToken)
#         if self.processType == ProcessType.DETECTION:
#            if paginationToken == None:
#                response = self.textract.get_document_text_detection(JobId=jobId, MaxResults=maxResults)
#            else:
#                response = self.textract.get_document_text_detection(JobId=jobId,MaxResults=maxResults, NextToken=paginationToken)
        blocks = response['Blocks']
        print('Detected Document Text')
        print('Pages: {}'.format(response['DocumentMetadata']['Pages']))
        # Display block information
#         for block in blocks:
#            self.DisplayBlockInfo(block)
#            print()
#            print()
        if 'NextToken' in response:
           paginationToken = response['NextToken']
        else:
            finished = True
#             return response
    # Put response on pages list
        pages.append(response)
        print('Document Detected.')

        if 'NextToken' in response:
            paginationToken = response['NextToken']
        else:
            finished = True

    # convert pages as JSON pattern
    pages = json.dumps(pages)
    return pages



In [ ]:
print(analyzePDF_results['DocumentMetadata'], 
     analyzePDF_results2['DocumentMetadata'], 
     analyzePDF_results3['DocumentMetadata'])

In [ ]:
analyzePDF_results

In [ ]:
analyzePDF_results2

In [ ]:
analyzePDF_results2.keys()

In [ ]:
analyzePDF_results2['NextToken']

In [ ]:
results_csv = get_csv(analyzePDF_results)

In [ ]:
analyzePDF_results_all= GetResults(JOBID)

In [ ]:
analyzePDF_results_all

In [ ]:
len(analyzePDF_results_all)

In [ ]:
analyzePDF_results_all.dtype()

In [ ]:
s3

In [ ]:
outputInJsonText= str(analyzePDF_results_all)
s3.put_object(Body=outputInJsonText, Bucket=BUCKET, Key='output-part1.json')
# pdfTextExtractionS3ObjectName= 

In [ ]:
outputInJsonText